In [0]:
#!pip install python-docx 
#!pip install deeppavlov

In [1]:
import deeppavlov
from deeppavlov.models.embedders.elmo_embedder import ELMoEmbedder
from nltk.tokenize import sent_tokenize, word_tokenize 
import warnings 
warnings.filterwarnings(action = 'ignore') 
import gensim 
from gensim.models import Word2Vec
import codecs
import numpy as np
import pandas as pd
import docx
import re
import string
import nltk
nltk.download("stopwords")
from nltk.corpus import stopwords
from pymystem3 import Mystem

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package perluniprops to /root/nltk_data...
[nltk_data]   Package perluniprops is already up-to-date!
[nltk_data] Downloading package nonbreaking_prefixes to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package nonbreaking_prefixes is already up-to-date!


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [6]:
from google.colab import drive
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


## Содержание.

**ТК состоит из 6 частей, 12 разделов, 62 глав и 424 статей(не считая подпункты, например Статья 330.3).** 

ЧАСТЬ 1.

* Раздел I.

Глава 1.(ст.1-14) Глава 2.(ст.15-22)

ЧАСТЬ 2.

* Раздел II.

Глава 3.(ст.23-28) Глава 4.(ст.29-34) Глава 5.(ст.35) 
Глава 6.(ст.36-39)Глава 7.(ст.40-51) Глава 8.(ст.52-53)
Глава 9.(ст.54-55)

ЧАСТЬ 3.

* Раздел III.

Глава 10.(ст.56-62) Глава 11.(ст.63-71) Глава 12.(ст.72-76)
Глава 13.(ст.77-84) Глава 14.(ст.85-90)

* Раздел IV. 

Глава 15.(ст.91-99) Глава 16.(ст.100-105)

* Раздел V. 

Глава 17.(ст.106-107) Глава 18.(ст.108-113) Глава 19.(ст.114-128)

* Раздел VI.

Глава 20.(ст.129-132) Глава 21.(ст.133-158) Глава 22.(ст.159-163)

* Раздел VII.

Глава 23.(ст.164-166) Глава 24.(ст.167-169) Глава 25.(ст.170-172) 
Глава 26.(ст.173-177) Глава 27.(ст.178-181) Глава 28.(ст.182-188)

* Раздел VIII.

Глава 29.(ст.189-190) Глава 30.(ст.191-195)

* Раздел VIX.

Глава 31.(ст.195.1-197) Глава 32.(ст.198-208)

* Раздел X.

Глава 33.(ст.209-210) Глава 34.(ст.211-215)
Глава 35.(ст.216-218) Глава 36.(ст.219-231)

* Раздел XI.

Глава 37.(ст.232-233) Глава 38.(ст.234-237)
Глава 39.(ст.238-250)

ЧАСТЬ 4.

* Раздел XII.

Глава 40.(ст.251-252) Глава 41.(ст.253-264) Глава 42.(ст.265-272)
Глава 43.(ст.273-281) Глава 44.(ст.282-288) Глава 45.(ст.289-292)
Глава 46.(ст.293-296) Глава 47.(ст.297-302) Глава 48.(ст.303-309) 
Глава 49.(ст.310-312) Глава 49.1.(ст.312.1-312.5) 
Глава 50.(ст.313-327) Глава 51.(ст.328-330) 
Глава 51.1.(ст.330.1-330.5) Глава 52.(ст.331-336) 
Глава 53.(ст.337-341) Глава 54.(ст.342-348) 
Глава 54-1.(ст.348.1-348.12) Глава 55.(ст.349-351.2)

ЧАСТЬ 5.

* Раздел XIII.

Глава 56.(ст.352) Глава 57.(ст.353-369) Глава 58.(ст.370-378)
Глава 59.(ст.379-380) Глава 60.(ст.381-398) Глава 61.(ст.399-418) 
Глава 62.(ст.419)

ЧАСТЬ 6.

* Раздел XIV.
(ст.420-424)

## Чтение ТК

In [0]:
document = docx.Document("drive/My Drive/LabourCode.docx")
text = []
chapter = ''
for paragraph in document.paragraphs:
    if paragraph.text!='':
        if paragraph.text.startswith('Статья') and chapter!='':
            text.append(chapter)
            chapter = ''
        if paragraph.text.endswith(('.',';',':')):
            chapter = chapter + paragraph.text
        else:
            chapter = chapter + paragraph.text + '. '

In [0]:
labourcode = text[1:]

In [0]:
pattern = re.compile(r"^Статья ([\d|\.]+)\. (.*)$")
data = {}
for article in labourcode:
    d = re.findall(pattern, article)
    if len(d)!=0:
        data[d[0][0]] = d[0][1]

In [0]:
lc = {'article №': list(data.keys()), 'text': list(data.values()), 'clean text': list(data.values())}

In [0]:
import collections

In [0]:
df = pd.DataFrame(lc)

## Preprocessing

In [12]:
#nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [0]:
def preprocess_text(text, mystem=Mystem(entire_input=False)):
    #russian_stopwords = stopwords.words("russian")
    punctuation = '!"#$%&()*+-/<=>?@[\]^_`{|}~'
    text = text.lower()
    for s in punctuation:
        text = text.replace(s, '')
    text = text.replace('[:;]', '.')
    tokens = nltk.word_tokenize(text)
    #tokens = [token for token in tokens if token not in russian_stopwords\
    #          and token != " "]
    #text = " ".join(tokens)
    return text

In [0]:
for i in range(len(df['clean text'])):
    df['clean text'][i] = preprocess_text(df['clean text'][i])

In [10]:
df.sample(10)

,article №,text,clean text
267,242,Полная материальная ответственность работника....,полная материальная ответственность работника....
269,244,Письменные договоры о полной материальной отве...,письменные договоры о полной материальной отве...
128,117,Ежегодный дополнительный оплачиваемый отпуск р...,ежегодный дополнительный оплачиваемый отпуск р...
362,327,Другие гарантии и компенсации. Гарантии и комп...,другие гарантии и компенсации. гарантии и комп...
302,274,Правовые основы регулирования труда руководите...,правовые основы регулирования труда руководите...
92,82,Обязательное участие выборного органа первично...,обязательное участие выборного органа первично...
258,233,Условия наступления материальной ответственнос...,условия наступления материальной ответственнос...
388,337,"Органы, направляющие работников в дипломатичес...","органы, направляющие работников в дипломатичес..."
27,27,Формы социального партнерства. Социальное парт...,формы социального партнерства. социальное парт...
439,359,Независимость государственных инспекторов труд...,независимость государственных инспекторов труд...


In [0]:
#import tensorflow.compat.v1 as tf
#tf.disable_v2_behavior()

In [11]:
elmo1 = ELMoEmbedder("http://files.deeppavlov.ai/deeppavlov_data/elmo_ru-wiki_600k_steps.tar.gz")

Using TensorFlow backend.


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [32]:
sent1 = elmo1([['вопрос', 'жизни', 'Вселенной', 'и', 'вообще', 'всего']])
sent1

array([[-0.22791581,  0.07285882, -0.4693267 , ...,  0.32489085,
         0.16321455, -0.15760408]], dtype=float32)

In [33]:
sent1.shape

(1, 1024)

In [34]:
sent2 = elmo1([['вопрос', 'жизни', ',', 'Вселенной', 'и', 'вообще', 'всего', '.']])
sent2

array([[-0.3336712 ,  0.13309552, -0.5554824 , ...,  0.29246017,
         0.12042639, -0.05358402]], dtype=float32)

In [0]:
import tensorflow as tf
import tensorflow_hub as hub
elmo = hub.Module("http://files.deeppavlov.ai/deeppavlov_data/elmo_ru-wiki_600k_steps.tar.gz", trainable=True)

In [0]:
def elmo_vector(x):
    embeddings = elmo([x], signature="default", as_dict=True)["elmo"]
    #x.tolist()
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        sess.run(tf.tables_initializer())
        # return average of ELMo features
        # 1 means: compute mean for 1 tensor(sentence)
        return sess.run(tf.reduce_mean(embeddings,1))

In [26]:
elmo_vector('вопрос жизни вселенной и вообще всего')

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


array([[-0.2599415 ,  0.08881932, -0.38537216, ...,  0.2884396 ,
         0.07476638, -0.21447879]], dtype=float32)

In [30]:
elmo_vector('вопрос жизни, вселенной и вообще всего')

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


array([[-0.21550333,  0.09075397, -0.3813815 , ...,  0.26505098,
         0.0987668 , -0.24937247]], dtype=float32)

In [27]:
elmo_vector('Вопрос жизни Вселенной и вообще всего')

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


array([[-0.3138059 ,  0.13712965, -0.40971747, ...,  0.3414384 ,
         0.3544406 , -0.0948571 ]], dtype=float32)

In [28]:
elmo_vector('Вопрос жизни, Вселенной и вообще всего!')

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


array([[-0.29900718,  0.10649759, -0.52120763, ...,  0.6242679 ,
         0.4439025 , -0.09051215]], dtype=float32)